In [1]:
#importamos librería
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
import sqlite3 as sql
# import xgboost as xgb


In [2]:
# Leemos el archivo CSV de actividad comercial
df_ca = pd.read_csv("data/commercial_activity_df.csv")

In [3]:
# Leemos el archivo CSV de productos
df_products = pd.read_csv("data/products_df.csv")


In [4]:
# Leemos el archivo CSV de datos sociodemográficos
df_sd = pd.read_csv("data/sociodemographic_df.csv")

In [5]:
# Eliminamos las columnas "Unnamed: 0" de los dataframes
del (df_ca["Unnamed: 0"])
del (df_products["Unnamed: 0"])
del (df_sd["Unnamed: 0"])

In [6]:
# Combinamos los dataframes df_ca y df_products utilizando las columnas 'pk_cid' y 'pk_partition' como claves
df_ca_products = pd.merge(df_ca, df_products, on=['pk_cid', 'pk_partition'], how='left')


In [7]:
# Combinamos los dataframes df_ca_products y df_sd utilizando las columnas 'pk_cid' y 'pk_partition' como claves
df_easy = pd.merge(df_ca_products, df_sd, on=['pk_cid', 'pk_partition'], how='left')


In [8]:
# Mostramos información del dataframe df_easy
df_easy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   entry_date          object 
 3   entry_channel       object 
 4   active_customer     float64
 5   segment             object 
 6   short_term_deposit  int64  
 7   loans               int64  
 8   mortgage            int64  
 9   funds               int64  
 10  securities          int64  
 11  long_term_deposit   int64  
 12  em_account_pp       int64  
 13  credit_card         int64  
 14  payroll             float64
 15  pension_plan        float64
 16  payroll_account     int64  
 17  emc_account         int64  
 18  debit_card          int64  
 19  em_account_p        int64  
 20  em_acount           int64  
 21  country_id          object 
 22  region_code         float64
 23  gender              object 
 24  age                 int6

In [9]:
# Convertimos la columna "pk_partition" a tipo datetime
df_easy["pk_partition"] = pd.to_datetime(df_easy["pk_partition"])


In [10]:
# Filtramos el dataframe df_easy para eliminar las filas donde la columna 'entry_date' sea '2015-02-29' o '2019-02-29'
# Estos son años donde no hay 29 de Febrero
df_easy = df_easy[(df_easy['entry_date'] != '2015-02-29') & (df_easy['entry_date'] != '2019-02-29')]
 

In [11]:
# Filtramos el dataframe df_easy para comprobar que no hay filas donde la columna 'entry_date' sea '2019-02-29'
df_easy[df_easy['entry_date'] == '2019-02-29']

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary


In [12]:
# Convertimos la columna "entry_date" a tipo datetime
df_easy["entry_date"] = pd.to_datetime(df_easy["entry_date"])


In [13]:
# Mostramos información del dataframe df_easy
df_easy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5956511 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   entry_date          datetime64[ns]
 3   entry_channel       object        
 4   active_customer     float64       
 5   segment             object        
 6   short_term_deposit  int64         
 7   loans               int64         
 8   mortgage            int64         
 9   funds               int64         
 10  securities          int64         
 11  long_term_deposit   int64         
 12  em_account_pp       int64         
 13  credit_card         int64         
 14  payroll             float64       
 15  pension_plan        float64       
 16  payroll_account     int64         
 17  emc_account         int64         
 18  debit_card          int64         
 19  em_account_p        int64         
 20  em_acou

In [14]:
# Creamos una copia del dataframe df_easy y la guardamos en df_easy_original
df_easy_original = df_easy.copy()

In [15]:
# Seleccionamos las columnas del dataframe df_easy que son de tipo int64 o int32
int_columns = df_easy.select_dtypes(include=['int64', "int32"]).columns

In [16]:
# Mostramos las columnas del dataframe df_easy que son de tipo int64 o int32
int_columns

Index(['pk_cid', 'short_term_deposit', 'loans', 'mortgage', 'funds',
       'securities', 'long_term_deposit', 'em_account_pp', 'credit_card',
       'payroll_account', 'emc_account', 'debit_card', 'em_account_p',
       'em_acount', 'age'],
      dtype='object')

In [17]:
# Convertimos las columnas seleccionadas a tipos de datos más eficientes
# Si el valor máximo de la columna es menor a 128, la convertimos a 'int16'
# De lo contrario, la convertimos a 'int32'
df_easy[int_columns] = df_easy[int_columns].apply(lambda col: col.astype('int16') if col.max() < 128 else col.astype('int32'))

In [18]:
# Mostramos información del dataframe df_easy
df_easy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5956511 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int32         
 1   pk_partition        datetime64[ns]
 2   entry_date          datetime64[ns]
 3   entry_channel       object        
 4   active_customer     float64       
 5   segment             object        
 6   short_term_deposit  int16         
 7   loans               int16         
 8   mortgage            int16         
 9   funds               int16         
 10  securities          int16         
 11  long_term_deposit   int16         
 12  em_account_pp       int16         
 13  credit_card         int16         
 14  payroll             float64       
 15  pension_plan        float64       
 16  payroll_account     int16         
 17  emc_account         int16         
 18  debit_card          int16         
 19  em_account_p        int16         
 20  em_acou

In [19]:
# Convertimos la columna 'active_customer' a tipo 'int16' para optimizar el uso de memoria
df_easy['active_customer'] = df_easy['active_customer'].astype('int16')

In [20]:
# Convertimos la columna 'region_code' a tipo 'object' para optimizar el uso de memoria
df_easy['region_code'] = df_easy['region_code'].astype('object')

In [21]:
# Mostramos información del dataframe df_easy para verificar los tipos de datos y el uso de memoria
df_easy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5956511 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int32         
 1   pk_partition        datetime64[ns]
 2   entry_date          datetime64[ns]
 3   entry_channel       object        
 4   active_customer     int16         
 5   segment             object        
 6   short_term_deposit  int16         
 7   loans               int16         
 8   mortgage            int16         
 9   funds               int16         
 10  securities          int16         
 11  long_term_deposit   int16         
 12  em_account_pp       int16         
 13  credit_card         int16         
 14  payroll             float64       
 15  pension_plan        float64       
 16  payroll_account     int16         
 17  emc_account         int16         
 18  debit_card          int16         
 19  em_account_p        int16         
 20  em_acou

In [22]:
# Guardamos los tipos de datos originales de las columnas seleccionadas en la variable original_dtypes
original_dtypes = df_easy[["entry_channel", "segment", "payroll", "pension_plan", "region_code", "gender"]].dtypes

In [23]:
# Mostramos las primeras filas del dataframe df_easy
df_easy.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01-28,2018-01-12,KHL,1,02 - PARTICULARES,0,0,0,0,...,0,0,0,1,ES,29.0,H,35,N,87218.10
1,1050611,2018-01-28,2015-08-10,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,13.0,V,23,N,35548.74
2,1050612,2018-01-28,2015-08-10,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,13.0,V,23,N,122179.11
3,1050613,2018-01-28,2015-08-10,KHD,0,03 - UNIVERSITARIO,1,0,0,0,...,0,0,0,0,ES,50.0,H,22,N,119775.54
4,1050614,2018-01-28,2015-08-10,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,50.0,V,23,N,NaN


In [24]:
# Guardamos el dataframe df_easy en un archivo pickle
df_easy.to_pickle("data/df_easy.pkl")

In [25]:
# Guardamos el dataframe df_easy en un archivo CSV
df_easy.to_csv('reports/df_easy.csv')